In [ ]:
# Install necessary libraries
!pip install -q autogluon[multimodal]


In [ ]:
import autogluon.core as ag
#from autogluon import ImagePredictor
import pandas as pd
import os
import numpy as np
from autogluon.multimodal import MultiModalPredictor


In [ ]:

from sklearn.model_selection import train_test_split

# Define path to the .npy files
image_folder = '/kaggle/input/beyond-visible-spectrum-ai-for-agriculture-2025/ot/ot/'

# Load CSV with the paths and labels
train_df = pd.read_csv('/kaggle/input/beyond-visible-spectrum-ai-for-agriculture-2025/train.csv')

# Function to load .npy files
def load_npy_image(image_name):
    image_path = os.path.join(image_folder, image_name)
    image = np.load(image_path)
    return image

# Load images and labels
X_train = []
y_train = []

for idx, row in train_df.iterrows():
    try:
        # Try to load and reshape the image
        image = load_npy_image(row['id'])
        
        # Check the shape of the image and reshape if necessary
        if image.shape == (128, 128, 125): 
            X_train.append(image)
            y_train.append(row['label'])
        else:
            print(f"Skipping {row['id']} due to unexpected shape {image.shape}")
    
    except ValueError as e:
        print(f"Error loading {row['id']}: {e}")
        continue 

# Convert the loaded data into NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Normalize the images if needed
X_train = X_train / np.max(X_train)

# Create a DataFrame for the training data (AutoGluon expects a tabular format for training)
train_data = pd.DataFrame({
    'image': list(X_train),
    'label': y_train
})

# Split dataset into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert data to AutoGluon-friendly format
train_data = pd.DataFrame({
    'image': list(X_train),
    'label': y_train
})

In [ ]:
#X_train.head()

In [ ]:
# Initialize AutoGluon MultiModalPredictor
predictor = MultiModalPredictor(label="label")

# Train the model on the hyperspectral data (use the same X_train and y_train format)
predictor.fit(train_data, time_limit=3600)  

In [ ]:
# Create a DataFrame for validation data
val_data = pd.DataFrame({
    'image': list(X_val),
    'label': y_val
})

# Evaluate the model on the validation data
validation_accuracy = predictor.evaluate(val_data)
print(f'Validation Accuracy: {validation_accuracy}')


In [ ]:
# Assuming test_data is available (it needs to be in the same format as train_data)
# Example: predict on new data
test_data = pd.DataFrame({
    'image': test_images,  # Replace with your actual test images
})

predictions = predictor.predict(test_data)

# Prepare the submission file
submission_df = pd.DataFrame({
    'id': test_data['id'],  # Replace with actual test IDs
    'label': predictions
})

submission_df.to_csv('submission.csv', index=False)
